In [15]:
import tensorflow as tf
import preprocessing
import numpy as np
import math

In [9]:
url = 'http://mattmahoney.net/dc/'
filename = preprocessing.maybe_download(url,'text8.zip', 31344016)
words = preprocessing.read_data(filename)
print('Data size', len(words))

('Found and verified', 'text8.zip')
('Data size', 17005207)


In [10]:
vocabulary_size = 50000
data, count, dictionary, reverse_dictionary = preprocessing.build_dataset(vocabulary_size,words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

('Most common words (+UNK)', [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)])
('Sample data', [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156], ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against'])


In [11]:
batch, labels = preprocessing.generate_batch(data, data_index,batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])


(3084, 'originated', '->', 12, 'as')
(3084, 'originated', '->', 5239, 'anarchism')
(12, 'as', '->', 3084, 'originated')
(12, 'as', '->', 6, 'a')
(6, 'a', '->', 12, 'as')
(6, 'a', '->', 195, 'term')
(195, 'term', '->', 6, 'a')
(195, 'term', '->', 2, 'of')


In [12]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


In [18]:
graph = tf.Graph()
with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()


In [30]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = preprocessing.generate_batch(data,data_index,batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()



Initialized
('Average loss at step ', 0, ': ', 271.1060791015625)
Nearest to was: hassan, creating, breakthroughs, subjunctive, ballista, mackintosh, yusupov, htm,
Nearest to who: timbres, populations, tsushima, enjoy, activision, trailers, ward, presley,
Nearest to four: ensued, macedonia, courted, scantily, wysiwyg, lovelock, unambiguously, stamped,
Nearest to history: lemming, pilgrims, libel, byrnes, frontal, comfortably, refugee, trait,
Nearest to such: romanum, osamu, yearbook, vanderbilt, roundly, travolta, kashima, uranium,
Nearest to this: ankh, bobby, dizzy, silicon, bidder, boitano, curly, nevado,
Nearest to eight: whitehead, pythagorean, meteor, splitting, antagonistic, catiline, eradicate, montessori,
Nearest to about: contexts, debilitating, protocol, creeping, inclusive, chant, middle, mand,
Nearest to UNK: kernel, shrews, columbus, hagiography, catastrophes, tsushima, emory, nahum,
Nearest to these: tropes, broz, designed, schottenheimer, archeological, surfacing, towne